# 1. EntropyMeasure

---

## 1.4 Classical Shadow - multiple experiments

Consider a scenario, you have multiple circuits that you want to run at once.

Call `.measure()` one by one will be inefficient,
no to mention that you also need to call `.anlyze()` for their post-processing.

Here we provide a more efficient way solve this problem,
where the true power of Qurrium as experiment manage toolkit.


### a. Import the instances


In [1]:
from qurry import ShadowUnveil

experiment_shadow = ShadowUnveil()

### b. Preparing quantum circuit

Prepare and add circuits to the `.wave` for later usage.


In [2]:
from qiskit import QuantumCircuit
from qurry.recipe import TrivialParamagnet, GHZ


def make_neel_circuit(n):
    qc = QuantumCircuit(n)
    for i in range(0, n, 2):
        qc.x(i)
    return qc


for i in range(2, 13, 2):
    experiment_shadow.add(TrivialParamagnet(i), f"trivial_paramagnet_{i}")
    experiment_shadow.add(GHZ(i), f"ghz_{i}")
    experiment_shadow.add(make_neel_circuit(i), f"neel_{i}")

experiment_shadow.waves

WaveContainer({
  'trivial_paramagnet_2': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7b9983694da0>,
  'ghz_2': <qurry.recipe.simple.cat.GHZ object at 0x7b99834f66c0>,
  'neel_2': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7b99830e3ef0>,
  'trivial_paramagnet_4': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7b9982f0ce60>,
  'ghz_4': <qurry.recipe.simple.cat.GHZ object at 0x7b9982f0cef0>,
  'neel_4': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7b99831520f0>,
  'trivial_paramagnet_6': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7b9982f0d220>,
  'ghz_6': <qurry.recipe.simple.cat.GHZ object at 0x7b9994957b30>,
  'neel_6': <qiskit.circuit.quantumcircuit.QuantumCircuit object at 0x7b9982f0d3a0>,
  'trivial_paramagnet_8': <qurry.recipe.simple.paramagnet.TrivialParamagnet object at 0x7b9982f0d6a0>,
  'ghz_8': <qurry.recipe.simple.cat.GHZ object at 0x7b9982f0d730>,
  'neel_8': <qiskit.circuit.quantumcircuit.Quantu

### c. Execute multiple experiments at once

Let's demonstrate the true power of Qurrium.


In [3]:
from qurry.qurrent.classical_shadow.arguments import ShadowUnveilMeasureArgs

Preparing a configuration list for multiple experiments with following parameters:

```python
class ShadowUnveilMeasureArgs(total=False):
    """Output arguments for :meth:`output`."""
    shots: int
    """Number of shots."""
    tags: Optional[tuple[str, ...]]
    """The tags to be used for the experiment."""

    wave: Optional[Union[QuantumCircuit, Hashable]]
    """The key or the circuit to execute."""
    times: int
    """The number of random unitary operator. 
    It will denote as `N_U` in the experiment name."""
    measure: Optional[Union[tuple[int, int], int, list[int]]]
    """The measure range."""
    unitary_loc: Optional[Union[tuple[int, int], int, list[int]]]
    """The range of the unitary operator."""
    unitary_loc_not_cover_measure: bool
    """Whether the range of the unitary operator is not cover the measure range."""
    random_unitary_seeds: Optional[dict[int, dict[int, int]]]
    """The seeds for all random unitary operator.
    This argument only takes input as type of `dict[int, dict[int, int]]`.
    The first key is the index for the random unitary operator.
    The second key is the index for the qubit.

    .. code-block:: python
        {
            0: {0: 1234, 1: 5678},
            1: {0: 2345, 1: 6789},
            2: {0: 3456, 1: 7890},
        }

    If you want to generate the seeds for all random unitary operator,
    you can use the function `generate_random_unitary_seeds` 
    in `qurry.qurrium.utils.random_unitary`.

    .. code-block:: python
        from qurry.qurrium.utils.random_unitary import generate_random_unitary_seeds
        random_unitary_seeds = generate_random_unitary_seeds(100, 2)
    """
```


In [4]:
config_list: list[ShadowUnveilMeasureArgs] = [
    {
        "shots": 1024,
        "times": 100,
        "wave": f"{wave_names}_{i}",
        "tags": (wave_names, f"size_{i}"),
    }
    for _ in range(10)
    for i in range(2, 13, 2)
    for wave_names in ["trivial_paramagnet", "ghz", "neel"]
]
print(len(config_list))

180


The `.multiOutput` will return an id of this `multimanager` instance,
which can be used to get the results and post-process them.

Each `multimanager` will export the experiments in a folder you can specify
by setting `save_location` parameter with default location for current directory
where Python executed.
It will create a folder with the name of the `multimanager` instance,
and inside it will create a folder for storing each experiment data.

It will do firstly in the building process, but you can skip it by setting `skip_build_write=True` to save time.
After all experiments are executed, it will export secondly,
which can also be skipped by setting `skip_output_write=True` for no files output.


In [5]:
multi_exps1 = experiment_shadow.multiOutput(
    config_list,
    summoner_name="qurshady",  # you can name it whatever you want
    multiprocess_build=True,
    # Using multiprocessing to build the experiments,
    # it will be faster but take all the CPU
    skip_build_write=True,
    # Skip the writing of the experiment as files during the build,
    save_location=".",
    # Save the experiment as files in the current directory
    multiprocess_write=True,
    # Writing the experiment as files using multiprocessing,
)
multi_exps1

| MultiManager building...
| Write "qurshady.001", at location "qurshady.001"


| 0/180   0%|          | - MultiManager building... - 00:00 < ?

| MultiOutput running...


| 0/180   0%|          | -  - 00:00 < ?

| Export multimanager...


...

| No quantity to export.
| Export multi.config.json for 4d59f946-8033-42e0-b6e7-2fdf6bc0108d


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurshady.001/qurryinfo.json...
| Exporting qurshady.001/qurryinfo.json done.


'4d59f946-8033-42e0-b6e7-2fdf6bc0108d'

In [6]:
experiment_shadow.multimanagers[multi_exps1]

<MultiManager(id="4d59f946-8033-42e0-b6e7-2fdf6bc0108d",
  name="qurshady.001",
  tags=(),
  jobstype="local",
  pending_strategy="tags",
  last_events={
    'output.001': '2025-05-21 22:56:09',},
  exps_num=180)>

### d. Run post-processing at once


In [7]:
experiment_shadow.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=True,
    multiprocess_write=False,
    selected_qubits=[0, 1],
)

| 0/180 - Analysis:  - 00:00 < ?

/home/nccu/uv-python-venv/quantumsphere12/lib/python3.12/site-packages/qurry/process/classical_shadow/classical_shadow.py:458: RuntimeWarning: The imaginary part of the trace of Rho square is not zero. The imaginary part is 1.2450456206139707e-36.method: einsum_aij_bji_to_ab_jax, numpy_precomputed, Python
  warnings.warn(


| "report.001" has been completed.


'4d59f946-8033-42e0-b6e7-2fdf6bc0108d'

In [8]:
print("| Available results:")
for k, v in (
    experiment_shadow.multimanagers[multi_exps1]
    .quantity_container["report.001"]
    .items()
):
    print("| -", k, "with length", len(v))

| Available results:
| - ('trivial_paramagnet', 'size_2') with length 10
| - ('ghz', 'size_2') with length 10
| - ('neel', 'size_2') with length 10
| - ('trivial_paramagnet', 'size_4') with length 10
| - ('ghz', 'size_4') with length 10
| - ('neel', 'size_4') with length 10
| - ('trivial_paramagnet', 'size_6') with length 10
| - ('ghz', 'size_6') with length 10
| - ('neel', 'size_6') with length 10
| - ('trivial_paramagnet', 'size_8') with length 10
| - ('ghz', 'size_8') with length 10
| - ('neel', 'size_8') with length 10
| - ('trivial_paramagnet', 'size_10') with length 10
| - ('ghz', 'size_10') with length 10
| - ('neel', 'size_10') with length 10
| - ('trivial_paramagnet', 'size_12') with length 10
| - ('ghz', 'size_12') with length 10
| - ('neel', 'size_12') with length 10


In [9]:
experiment_shadow.multimanagers[multi_exps1].quantity_container["report.001"][
    ("trivial_paramagnet", "size_10", "system_range_5")
][:2]

[]

### e. Run post-processing at once with specific analysis arguments

At first, we need to get the each experiment's id in the `multimanager` instance.


In [10]:
expkeys_of_multi_exps1 = list(
    experiment_shadow.multimanagers[multi_exps1].exps.keys()
)
print(len(expkeys_of_multi_exps1))

180


1. If you want to run the post-processing for some specific experiments,
   for example, the first 3 experiments we get for the `multimanager` instance.


In [11]:
experiment_shadow.multiAnalysis(
    summoner_id=multi_exps1,
    analysis_name="first_3",
    skip_write=True,
    multiprocess_write=False,
    specific_analysis_args={k: {
        "selected_qubits": [0, 1],   
    } if idx < 3 else False for idx, k in enumerate(expkeys_of_multi_exps1)},
)

| 0/180 - Analysis:  - 00:00 < ?

| "first_3.002" has been completed.


'4d59f946-8033-42e0-b6e7-2fdf6bc0108d'

In [12]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_shadow.multimanagers[multi_exps1]
        .quantity_container["first_3.002"]
        .values()
    ),
)

| Available results:
| length: 3


2. Or manually specify all the analysis arguments for each experiment.


In [13]:
experiment_shadow.multiAnalysis(
    summoner_id=multi_exps1,
    skip_write=False,
    analysis_name="all_manual",
    multiprocess_write=True,
    specific_analysis_args={k: {
        "selected_qubits": [0, 1],  # selected qubits for the analysis
    } for idx, k in enumerate(expkeys_of_multi_exps1)},
)

| 0/180 - Analysis:  - 00:00 < ?

| "all_manual.003" has been completed.
| Export multimanager...


| 0/9 - Exporting MultiManager content... - 00:00 < ?

| 0/3 - exporting quantity - 00:00 < ?

| Export multi.config.json for 4d59f946-8033-42e0-b6e7-2fdf6bc0108d


| 0/179 - Exporting experiments... - 00:00 < ?

| 0/180 - Loading file infomation... - 00:00 < ?

| Exporting file taglist...
| Exporting qurshady.001/qurryinfo.json...
| Exporting qurshady.001/qurryinfo.json done.


'4d59f946-8033-42e0-b6e7-2fdf6bc0108d'

In [14]:
print("| Available results:")
print(
    "| length:",
    sum(
        len(v)
        for v in experiment_shadow.multimanagers[multi_exps1]
        .quantity_container["all_manual.003"]
        .values()
    ),
)

| Available results:
| length: 180


### f. Read exported multimanager data


In [15]:
multi_exps1_reades = experiment_shadow.multiRead(
    save_location=".",
    summoner_name="qurshady.001",
)

| Retrieve qurshady.001...
| at: qurshady.001


| 0/180   0%|          | - Loading 180 experiments ... - 00:00 < ?

In [16]:
expkeys_of_multi_exps1

['5cd5d9a5-bc30-4f13-aa6c-9f30ffd18b58',
 '874e49e3-eef8-45f4-b6e2-6f68f8d9f693',
 '807219a8-0a07-45f9-b84e-3f1eeeddf9e2',
 'f9cb1b03-3c1e-404a-8416-6eb3665ea601',
 'b3cce98d-d1bc-4a7e-b1d1-897503d08b9c',
 'ff0f0358-1e80-4c95-8535-d02908ec11ee',
 '6ed79b74-482d-4922-ab33-b8646cb31449',
 '35af5768-8725-4e4a-b5b5-db85a1c5c0cc',
 'a4bce006-6b74-4680-b76c-7a9fa6b9c6db',
 '7affa9bf-ba20-4eb3-95bd-524dd057fddb',
 'd522ceb0-8823-4109-80d0-74a48326943a',
 '4501121b-f202-41e3-b7a2-72d4fd00aea8',
 'c3d13b20-f222-44a1-a117-6104bf6da0ee',
 'bae4f29c-2d13-44e9-aec6-c3ae03c6f933',
 '8a779f97-4266-457e-9cc8-4294ac28e5a9',
 '6e86c449-671e-4c0a-80c8-80678ac93961',
 'eb7c74e7-79bf-4fd1-9bb2-94fd27b32922',
 '5a67a918-47b5-4b58-a5b2-3e169b9d996a',
 'f8f0313a-1b99-4a9f-a801-7f5fa679d2fc',
 '64f07202-1b17-47c5-a7d3-d99de9219bb2',
 '8f49e01d-0c3c-4042-bbd9-acec1b8c4d9a',
 '6709b1f2-5a60-417d-9bbd-8114a33c2a5c',
 '1b74ab27-b405-4522-8721-b25be3cc1b84',
 '5f130f91-6d32-465c-b63e-c190bb95be68',
 '2f767ae9-5552-

---

### Post-Process Availablities and Version Info

In [17]:
from qurry.process import AVAIBILITY_STATESHEET
AVAIBILITY_STATESHEET

 | Qurry version: 0.12.2.dev1
---------------------------------------------------------------------------
 ### Qurry Post-Processing
   - Backend Availability ................... Python Cython Rust   JAX   
 - randomized_measure
   - entangled_entropy.entropy_core_2 ....... Yes    Depr.  Yes    No    
   - entangle_entropy.purity_cell_2 ......... Yes    Depr.  Yes    No    
   - entangled_entropy_v1.entropy_core ...... Yes    Depr.  Yes    No    
   - entangle_entropy_v1.purity_cell ........ Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_core_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap.echo_cell_2 ....... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_core ...... Yes    Depr.  Yes    No    
   - wavefunction_overlap_v1.echo_cell ...... Yes    Depr.  Yes    No    
 - hadamard_test
   - purity_echo_core ....................... Yes    No     Yes    No    
 - magnet_square
   - magnsq_core ............................ Yes    No     No     No  